In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
sns.set_palette("colorblind")

In [26]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential   #, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Rescaling, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from tensorflow.keras.initializers import GlorotUniform

# Personalized Fashion Recommendations
## Baseline model for image recognition

The images are RGB color coded.

### Load and preprocess the images

In [43]:
# List of image paths
image_paths = ["samples/sample_tshirt.jpg", "samples/sample_tshirt.jpg"]

# List to store the loaded images
images = []

In [32]:
TARGET_SIZE = (100, 100)

In [44]:
# Load the images and convert them to numpy arrays
for path in image_paths:
    image = load_img(path=path, target_size=TARGET_SIZE)
    image = img_to_array(image)
    images.append(image)

# Convert the list of images to a numpy array
images = np.stack(images, axis=0)

In [47]:
images.shape

(2, 100, 100, 3)

In [59]:
# Fake labels
labels = np.random.choice(range(NUMBER_OF_CLASSES), size=len(images), replace=True)
labels.shape

(2,)

### Train-test split

In [84]:
#images_train, images_test, labels_train, labels_test = train_test_split(images, labels, train_size=2)

images_train = images
labels_train = labels

### Model building

In [48]:
NUMBER_OF_CLASSES = 10

In [50]:
# Define the model
baseline_model = Sequential(layers=[
    # Rescale the image in the [0, 255] range to be in the [0, 1] range
    Rescaling(scale=1./255, input_shape=images.shape[1:]),
    
    # The first convolutional layer
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    
    # The max pooling layer
    MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten the output
    Flatten(),
    
    # Fully connected layer
    Dense(units=32, activation='relu'),
    
    # The output layer with units=NUMBER_OF_CLASSES. Sum of outputs equals to 1.
    Dense(NUMBER_OF_CLASSES, activation='softmax')
], name="baseline_model")

2023-02-06 18:17:57.865728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [66]:
# Check if the input and output of the model are correct
assert baseline_model.input_shape[1:] == images.shape[1:]
assert baseline_model.output_shape[1:] == (NUMBER_OF_CLASSES,)

In [87]:
# Compile the model
baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [88]:
# Make Early Stopping and Checkpoints
model_checkpoint = ModelCheckpoint(filepath='best_model_baseline_CNN.h5',
                                   monitor='val_loss',
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='min',
                                   verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')

### Model fitting

In [89]:
baseline_model.fit(x=images_train, y=labels_train, batch_size=1, epochs=100, validation_split=0.1,
          callbacks=[early_stop, model_checkpoint])

Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 2.1539 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 19.42424, saving model to best_model_baseline_CNN.h5
1/1 [==============================] - 1s 853ms/step - loss: 2.1539 - accuracy: 0.0000e+00 - val_loss: 19.4242 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 0.0047 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 19.42424
1/1 [==============================] - 0s 35ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 34.6176 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 2.8014e-05 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 19.42424
1/1 [==============================] - 0s 34ms/step - loss: 2.8014e-05 - accuracy: 1.0000 - val_loss: 46.8243 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 3.5763e-07 - accuracy: 1.0000
Epoch 4: val_loss di

In [91]:
# !git branch
!git add .
# !git commit -m "EDA of article.csv update"
# !git push origin kuzma

In [92]:
!git status

On branch kuzma
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .gitignore
	modified:   EDA_kuzma.ipynb
	new file:   baseline_model_CNN_kuzma.ipynb
	new file:   best_model_baseline_CNN.h5

